# Premier League Football : analysis on overall team performance over 27 seasons. 

-Introduction/contex-
Over the past decade, the way of analyzing a performance of sports players or teams has shifted to more on numbers. 
It became more com
nu,bers numbers numbers ....etc etc etc 

Analysis on this project will focus on performance of each team rather than performance of individual players. The analysis will be from simple stats to some complex stats. For instance, number of goals, wins and average number of free kicks per match. The details of what is being a target of analysis are within this notebook. Basic stats are still capable to show the performance and possibiliy a some charactristsics of each team over the past 27 seasons.  
The final output of analysis should be easy to understand by wide range of audeince, therefore it must be visualized in a form of graph or table and should not require some special knowledge of mathmatics. HOwever, it will require some basic knowledge on football.    

The source data for this project is all match results from 1993/1994 season to present. Format of source data is csv and single csv file contains a match result of one season. 

any weaknesses or potential caveats in your approach

# Premier League match result data

I was originally looking for a one data that contains all the result from 1993/1994 season to 2019/2020 season. However, I could only find indivisual CSV files for each seasons. At the begginning, I was going to download eachcsv files manually, but it is much effcient to scrape them from the web page. I scrrape all data set from the following web page http://www.football-data.co.uk/englandm.php using BeautifulSoup. Initially, each csv files are scraped individually, then they will be combine to two different csv files.

I notice that data after 2000/2001 season has more stats compare to seasons before 2000/2001 season. Data before 2000/2001 season contains only a match results where seasons after 2000/2001 contains more details stats of each match such as number of corner kicks, free kicks, and etc.

In [24]:
from bs4 import BeautifulSoup
import requests
import re 

headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'}

In [25]:
# Webpage that I scrape CSV files from.
url = "http://www.football-data.co.uk/englandm.php"
all_files = [] #this list contains names of each csv files
# download all csv files from the web page 
with requests.Session() as req:
    r = req.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    for link in soup.find_all(href=re.compile('E0.csv')):
        file = link.get('href')
        new_link = "http://www.football-data.co.uk/" + file
        r = req.get(new_link)
        name = file.rsplit("/", 2)[-2]
        name = name + ".csv"
        all_files.append(name)
        with open(name, 'wb') as f:
            f.write(r.content)
all_files.reverse() 

all_files is a list that contains name of each csv files, and this list start with 2020/2021 season. But it makes more sence to have them in past to present order. 

In [26]:
# group all csv files into two different group 
file_group1 = []
file_group2 = []
for i in range(len(all_files)):
    if i < 7:
        file_group1.append(all_files[i])
    else:
        file_group2.append(all_files[i])

print("93/94 season to 99/00 season") 
print(file_group1)
print("00/01 season to present") 
print(file_group2)

93/94 season to 99/00 season
['9394.csv', '9495.csv', '9596.csv', '9697.csv', '9798.csv', '9899.csv', '9900.csv']
00/01 season to present
['0001.csv', '0102.csv', '0203.csv', '0304.csv', '0405.csv', '0506.csv', '0607.csv', '0708.csv', '0809.csv', '0910.csv', '1011.csv', '1112.csv', '1213.csv', '1314.csv', '1415.csv', '1516.csv', '1617.csv', '1718.csv', '1819.csv', '1920.csv', '2021.csv']


I grouped files based on similarity of data that are contain in each files.  As I mnetioned, first group contains less data about each match than second group of files. By separating them into two, I can do more analysis on second group since there are only limited possibility of analysis on first group.  


The following code will manipulate data by modify and combine csv files in order to carry out analysis. 

In [27]:
import os, glob
import pandas as pd
import numpy as np

In [28]:
# modify some files that contains unncessary blank rows 
def modify_file(file):
    df = pd.read_csv(file)
    df['Div'].replace(' ', np.nan, inplace=True)
    df.dropna(subset=['Div'], inplace=True)
    df.to_csv(file, index=False)
    
for file in file_group1:
    modify_file(file)

I notice that files in group 1 contains unncessary blank rows at the end of each file, and function modify_file( ) remove those blank rows. Merging those files without modifying them will cause huge gaps within the merged file and it could cause some errorswhile data annalysis. 

In [29]:
# merge group 1 into one file called "9300.csv" 
df_group1 = (pd.read_csv(f) for f in file_group1)
df_group1_merged = pd.concat(df_group1, sort=False)
df_group1_merged.to_csv("9300.csv")

# merge group 2 into one file called "0021.csv"
df = pd.read_csv("1920.csv")
header = list(df.columns.values)
df_group2 = (pd.read_csv(f, names=headers, encoding='latin1') for f in file_group2)
df_group2_merged = pd.concat(df_group2, names = headers)
df_group2_merged.to_csv("0021.csv")

In this case, it is ideal to define one function and use it to combine csv files to avoide repittion of code. However, second group of files contains more columns and thererfore it require a bit different approcah. 

In [30]:
# delete unnecessary files from project 
total_file_num = len(all_files)
for i in range(0, total_file_num):
    os.remove(all_files[i])

At this point, I have two files that both contain data about EPL matches with second file contains more stats. I will carry two types of analysis: one for both file and another one for just second file (0021.csv) 